# <font color='purple'>Исследование надёжности заёмщиков</font>

1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропущенных значений](#null)
    * [Замена типов](#types)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemmas)
    * [Категоризация](#categorys)
3. [Ответы на вопросы](#answers)
4. [Вывод](#end)

## Открытие данных<a id="start"></a>

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

from pymystem3 import Mystem
m = Mystem()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Предобработка данных<a id="preprocessing"></a>

### Обработка пропущенных значений<a id="null"></a>

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()
# Посчитаем данные с минусами реальными и приведем к нормальному виду.
# И вот это на будущее захватим.
print(data['days_employed'].mean())
print(data['total_income'].mean())
# Есть мысль заменить пропуски на средние значения.
# Но сначала разберем имеет ли это смысл, сравнив количество рабочих дней с возрастом.
data['work_years'] = data['dob_years'] - 18 # Примем отсчет рабочего стажа с 18 лет.
data['days_of_years'] = data['days_employed'] / 365 / data['work_years']
# data[data['days_of_years'] <= 1]
# Мы видим, что помимо пропущенных данных у нас еще порядка 4000 некорректных, что составляет около 20%,
# а значит заменять пропущенные значения на средние будет неправильно, как минимум чтоб ответить на вопрос влияния дохода
# на кредит. Более логично будет их удалить чтоб не портить влияние на иные строки.
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
# Прогоним через медиану, тогда надо снова обновить значения.
data['work_years'] = data['dob_years'] - 18
data['days_of_years'] = data['days_employed'] / 365 / data['work_years']

66914.72890682236
167422.30220817294


## Замена типов<a id="types"></a>

In [5]:
data['days_employed'] = data['days_employed'].astype('int64')
data['total_income'] = data['total_income'].astype('int64')
data.info()
# Выбраны эти столбцы, так как потеря десятой доли числа ничего не изменит.
# Если понадобится, новый столбец тоже можно перезаписать или изменить тип, но это не имеет значения.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
work_years          21525 non-null int64
days_of_years       21525 non-null float64
dtypes: float64(1), int64(8), object(5)
memory usage: 2.3+ MB


## Обработка дубликатов<a id="dublicates"></a>

In [6]:
data.duplicated().sum()

54

In [7]:
# Видим на выходе 0, скорее всего потому что дубликаты если и были, то в некорректных данных.
# Если бы они были, применили бы строку data.drop_duplicates().reset_index(drop = True) для их удаления.
# Причины могут быть любые, но самое вероятное в данном случае повторное обращения клиента и заведение заявки (если индексы
# далеко друг от друга) или же ошибка оператора (если индексы идут подряд).

<font color='purple'>Окей, теперь дубликаты у нас имеются, значит от них и избавимся.</font>

In [8]:
data = data.drop_duplicates().reset_index(drop = True)

In [9]:
data['education']= data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
# Создаем себе комфорт

In [10]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [11]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

<font color='green'>С дубликатами разобрались, но не забудем проверить, не появятся ли они после заполнения пропусков.

## Лемматизация<a id="lemmas"></a>

In [12]:
# А вот тут уже начнем производить лемматизацию
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [13]:
data['lemmas'] = data['purpose'].apply(m.lemmatize)
# Здесь имеем один минус - в данных фигурирует фраза "жилая недвижимость",
# а в моих планах категоризации жилье и недвижимость это разные категории.
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,work_years,days_of_years,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,24,0.963205,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,18,0.612603,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15,1.027109,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,14,0.807191,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,35,26.635309,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,25,0.496363,"[операция, , с, , жилье, \n]"
21467,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,49,19.230495,"[сделка, , с, , автомобиль, \n]"
21468,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,20,0.289500,"[недвижимость, \n]"
21469,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,20,0.426367,"[на, , покупка, , свой, , автомобиль, \n]"


## Категоризация<a id="categorys"></a>

In [14]:
from collections import Counter as c
lemm = m.lemmatize(' '.join(data['purpose'].unique()))
lemm
c(lemm).most_common()

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

<font color="purple">Ну да, часок убил, но вроде понятно

In [15]:
def cat(row):
        if 'образование' in row:
            return 'образование'
        if 'жилье' in row:
            return 'жилье'
        if 'недвижимость' in row:
            return 'недвижимость'
        if 'автомобиль' in row:
            return 'автомобиль'
        if 'свадьба' in row:
            return 'свадьба'

In [16]:
data['category'] = data['lemmas'].apply(cat)

In [17]:
data['category'].value_counts()

недвижимость    6353
жилье           4461
автомобиль      4308
образование     4014
свадьба         2335
Name: category, dtype: int64

In [18]:
fam = data[['family_status_id', 'family_status']]
fam = fam.drop_duplicates().reset_index(drop = True)
fam

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


## Ответы на вопросы<a id="answers"></a>

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [19]:
data_children = data.pivot_table(index=['children'], values='debt', aggfunc='mean')
data_children['debt'] = data_children['debt'] * 100

In [20]:
data_children.round(2).astype('str') + '%'

,debt
children,
0,7.54%
1,9.16%
2,9.45%
3,8.18%
4,9.76%
5,0.0%
20,10.53%


**Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [21]:
data_family = data.pivot_table(index=['family_status'], values='debt', aggfunc='mean')
data_family['debt'] = data_family['debt'] * 100

In [22]:
data_family.round(2).astype('str') + '%'

,debt
family_status,
в разводе,7.11%
вдовец / вдова,6.57%
гражданский брак,9.32%
женат / замужем,7.54%
не женат / не замужем,9.75%


In [23]:
data_family = data.pivot_table(index=['family_status'], values=['debt','dob_years'], aggfunc='mean')
data_family['debt'] = data_family['debt'] * 100
data_family['debt'] = data_family['debt'].round(2).astype('str') + '%'
data_family['dob_years'] = data_family['dob_years'].astype('int64')

In [24]:
data_family

,debt,dob_years
family_status,,
в разводе,7.11%,45
вдовец / вдова,6.57%,56
гражданский брак,9.32%,42
женат / замужем,7.54%,43
не женат / не замужем,9.75%,38


**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [26]:
# А вот тут придется немного упростить задачу

def income(summ):
        if summ < 50000:
            return '< 50'
        if summ >= 50000 and summ < 100000:
            return 'between 50 and 100'
        if summ >= 100000 and summ < 150000:
            return 'between 100 and 150'
        if summ >= 150000 and summ < 250000:
            return 'between 150 and 250'
        return '> 250'

data.quantile(q=[0.25,0.5,0.75,1])

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,work_years,days_of_years
0.25,0.0,1023.5,33.0,1.0,0.0,0.0,107654.5,15.0,0.139111
0.50,0.0,1996.0,42.0,1.0,0.0,0.0,142594.0,24.0,0.309694
0.75,1.0,5329.5,53.0,1.0,1.0,0.0,195767.5,35.0,0.749055
1.00,20.0,401755.0,75.0,4.0,4.0,1.0,2265604.0,57.0,229.290589


In [27]:
data['cat_income'] = data['total_income'].apply(income)
data['cat_income'].value_counts()

between 100 and 150    7175
between 150 and 250    7020
between 50 and 100     4091
> 250                  2813
< 50                    372
Name: cat_income, dtype: int64

In [28]:
data_income = data.pivot_table(index=['cat_income'], values='debt', aggfunc='mean')

In [29]:
data_income

,debt
cat_income,
< 50,0.061828
> 250,0.068966
between 100 and 150,0.086969
between 150 and 250,0.081054
between 50 and 100,0.080909


А вот тут очень даже вроде бы как странно. При разном доходе почти одинаковое отсутствие долгов. Значит порядочные граждане.

**Как разные цели кредита влияют на его возврат в срок?**

In [30]:
data_purpose = data.pivot_table(index=['category'], values='debt', aggfunc='mean')

In [31]:
data_purpose

,debt
category,
автомобиль,0.093547
жилье,0.069043
недвижимость,0.074610
образование,0.092177
свадьба,0.079657


И снова похожие цифры. Разные цели кредита почти никак не влияют на долги, люди возвращают кредиты. Какие платежеспособные клиенты, однако.

# Вывод<a id="end"></a>

Используя полученные данные мы смело можем вывести такой вывод:
 - На количество детей можно не обращать внимание.
 - Самыми платежеспособными оказались семейные пары или разведенные, а так же вдовцы. Разбирая доходность, наиболее выгодным будет давать кредиты клиентам с низким или очень высоким доходом.